In [17]:
import pandas as pd

import requests
from bs4 import BeautifulSoup
import re

import warnings
warnings.filterwarnings("ignore")

## Web Scraping for El youm7

In [18]:
def get_article_details(url):

    data = requests.get(url).text
    soup = BeautifulSoup(data)
    title = soup.title.text

    image = soup.find("div", attrs={"class": "img-cont"}).find("img")["src"]

    writer = (
        soup.find("span", attrs={"class": "writeBy"})
        .text.replace("كتبت", "")
        .replace("كتب", "")
        .strip()
    )
    tags = soup.find("meta", attrs={"name": "keywords"})["content"]

    article = soup.find("div", attrs={"id": "articleBody"}).text
    article = re.findall('[\w\._,"]+', article)
    article = (" ").join(article)


    return {
        "url": url,
        "title": title,
        "image": image,
        "writer": writer,
        "tags": tags,
        "article": article,
    }

In [19]:
def get_links_from_page(url,page_no):
    data = requests.get(
         url
        + str(page_no)
        + ""
    ).text
    soup = BeautifulSoup(data)
    links = []
    for a in soup.find("div", attrs={"id": "paging"}).findAll("a"):
        links.append("https://youm7.com" + a["href"])
    return list(set(links))

In [ ]:

def get_article_details_by_news_type(news_type_url,number_of_pages):
    df = pd.DataFrame()

    for i in range(1, number_of_pages+1):   # for the list of pages links
        # print(f'Getting page {i}')
        
        for link in get_links_from_page(news_type_url,i):   # for the Pages details
        # for link in get_links_from_page(news_type_url,i)[:5]:   # for test the Pages details 
            # print(f'Getting links from Page  {link}')
            
            new_df = pd.DataFrame(get_article_details(link), index=[0])
            df = pd.concat([df, new_df], ignore_index=True)
    return df        

In [21]:
def get_all_articles_links(url):
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    links = list(map(lambda x: x.find("a")["href"] if x.find("a") else None, soup.find_all("h2")))
    filtered_links = [link for link in links if link is not None]
    filtered_links


    for i in range(0, len(filtered_links)):
        filtered_links[i] = "https://youm7.com" +filtered_links[i]
        
    return filtered_links
    

In [22]:
url = "https://www.youm7.com/"

all_articles_links = get_all_articles_links(url)

# all_articles_links = all_articles_links[:2]  # use only to test this code

all_df =[]
for articles_link in all_articles_links:
    df =  get_article_details_by_news_type(articles_link,1)
    all_df.append(df)

In [23]:
all_df[1]

,url,title,image,writer,tags,article
0,https://youm7.com/story/2024/12/2/وزير-الخارجي...,وزير الخارجية: تهجير الفلسطينيين وتصفية القضية...,https://img.youm7.com/large/20241128124701471.jpg,أحمد جمعة- هناء أبو العز,"وزير الخارجية,فلسطين,مصر,غزة,اسرائيل,القضية ال...",أكد بدر عيد العاطي وزيرة الخارجية والهجرة أن ت...
1,https://youm7.com/story/2024/12/2/وزير-الخارجي...,وزير الخارجية والهجرة يبحث مع نائب وزير خارجية...,https://img.youm7.com/large/202412020614151415...,أحمد جمعة,"وزير الخارجية والهجرة,وزير خارجية استراليا,است...",التقى الدكتور بدر عبد العاطي وزير الخارجية وال...
2,https://youm7.com/story/2024/12/2/وزير-الخارجي...,وزير الخارجية يطالب إسرائيل بالانسحاب من الجان...,https://img.youm7.com/large/20241202040727727.jpg,محمد عبد العظيم,"وزير الخارجية,مؤتمر وزير الخارجية,بدر عبد العا...",قال الدكتور بدر عبد العاطي وزير الخارجية والهج...
3,https://youm7.com/story/2024/12/2/مدبولى-يلتقي...,"مدبولى يلتقي رئيس مجلس إدارة شركة ""انكا"" العال...",https://img.youm7.com/large/202412020447134713...,هند مختار,"شركة انكا العالمية,الحكومة,مصطفى مدبولى,الاستث...",التقى الدكتور مصطفى مدبولي رئيس مجلس الوزراء م...
4,https://youm7.com/story/2024/12/2/أخبار-مصر-غد...,أخبار مصر.. غدا طقس معتدل نهارا وأمطار والعظمى...,https://img.youm7.com/large/202410311053145314...,,"الرئيس السيسي,السيسي,عبد الفتاح السيسي,اخبار ا...",نشر اليوم السابع على مدار الساعات القليلة الما...
